In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from src.config import DATA_DIRECTORY
from src import utils

def get_last_ended_action(group):
    """Return the enforcement action with the most recent VIOL_END_DATE"""
    return group.sort_values(by='VIOL_END_DATE', na_position='first', ascending=False).iloc[0]


pd.set_option("display.max_columns", 999)

ModuleNotFoundError: No module named 'src'

# Load the table of compliance violations

In [ ]:
df = pd.read_csv(DATA_DIRECTORY / 'interim' / 'hr2w_exceedance.csv')

### A sample of violations

In [ ]:
df.sample(10)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
df.ENF_ACTION_TYPE_ISSUED.value_counts().plot.bar(title='Enforcement action type issued', ax=axs[0]);
df.ANALYTE_NAME.value_counts().plot.bar(title='Analyte name', ax=axs[1]);

In [ ]:
df.ENF_ACTION_TYPE_ISSUED.value_counts()

In [ ]:
groups = df.groupby(['WATER_SYSTEM_NUMBER', 'ANALYTE_NAME'])

last_ended_action = groups.apply(get_last_ended_action)

Do not include water system/analyte combinations that ended with a return to compliance

In [ ]:
active_violations = last_ended_action[last_ended_action.ENF_ACTION_TYPE_ISSUED != 'RETURN TO COMPLIANCE']
active_violations.reset_index(drop=True, inplace=True)

# There are 330 water systems with at least one analyte out of compliance

In [ ]:
print(f"{active_violations.WATER_SYSTEM_NUMBER.unique().size} water systems with at least one analyte out of compliance")

We also compute a histogram of number of water systems with different numbers of analytes out of compliance.

In [ ]:
fig, ax = plt.subplots();
counts = active_violations.groupby('WATER_SYSTEM_NUMBER').apply(lambda x: len(x)).sort_values(ascending=False)
counts.hist(bins=np.arange(1, max(counts) + 1, 1));
ax.set_xlabel('Number of analytes out of compliance');
ax.set_ylabel('Number of water systems');

In [41]:
population_affected = active_violations.drop_duplicates(subset=['WATER_SYSTEM_NUMBER']).POPULATION.sum()
print(f"{utils.millify(population_affected)} people affected")

1 million people affected
